In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    'cookiesession1': '678B28F695C90B262BFC00B79BBD6CC1',
    'lhc_per': 'vid|a3d85efe963899fee2b5',
    '_gid': 'GA1.3.289911229.1685154797',
    '_ga': 'GA1.1.2075372866.1684772919',
    '_ga_3C534XGNC2': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_Q33BVTTDL1': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_QHEB12PC6D': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'cookiesession1=678B28F695C90B262BFC00B79BBD6CC1; lhc_per=vid|a3d85efe963899fee2b5; _gid=GA1.3.289911229.1685154797; _ga=GA1.1.2075372866.1684772919; _ga_3C534XGNC2=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_Q33BVTTDL1=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_QHEB12PC6D=GS1.1.1685154797.4.1.1685154882.0.0.0',
    'Pragma': 'no-cache',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [4]:
def download(url, file_name, retry = 3):
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360, cookies=cookies, headers=headers)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [5]:
files = glob('pages/*.json')
len(files)

18787

In [6]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [7]:
filtered_pdfs = []
for p in tqdm(pdfs):
    if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
        continue
    if not p.endswith('.pdf'):
        continue
    if 'ade.sagepub.com' in p:
        continue
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|███████████████████████████████| 235433/235433 [00:00<00:00, 687275.59it/s]


235130

In [10]:
uum = [f for f in filtered_pdfs if 'utm.my' in f]
len(uum)

6

In [11]:
uum

['http://gmm.fsksm.utm.my/~cogramm/cgi-bin/2004/upload_files/abstract/449317633605827_1.pdf',
 'http://web.utm.my/ipasa/images/stories/MJCE/2004/vol_16_no_2/Application%20of%20Artificial%20Neural%20Network%20Models%20for%20Predicting%20Water%20Quality%20Index.pdf',
 'http://www.fp.utm.my/medc/journals/vol2/11.%20WHAT%20PREDICTS%20ATTITUDES%20TOWARD%20COMPUTER.pdf',
 'http://www.fs.utm.my/matematika/images/stories/matematika/200521101.pdf',
 'http://www.fs.utm.my/matematika/images/stories/matematika/20082423.pdf',
 'http://www.penerbit.utm.my/onlinejournal/46/E/JT46EJUN2.pdf']

In [14]:
# !rm -rf uitm
!mkdir uum

In [ ]:
max_worker = 1

uitm = sorted(list(uum))
for i in tqdm(range(1, len(uum), max_worker)):
    urls = [(url, os.path.join('uum', f'{i}-{no}.pdf')) for no, url in enumerate(uitm[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  0%|                                     | 13/45796 [00:36<35:14:31,  2.77s/it]

In [ ]:
uitm[23191]